### Initial Imports:

In [45]:
from pathlib import Path
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", 300)

import requests
from dotenv import load_dotenv
from datetime import datetime,timedelta
import math

import json
from web3.auto import w3

In [46]:
import alpaca_trade_api as tradeapi
import os
import time

load_dotenv()

True

### Alpaca Setup:

In [47]:
ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")
 
#API endpoint URL
url = "https://paper-api.alpaca.markets"
api = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, url, api_version='v2')

### Additional Installation:

In [3]:
# !pip install yfinance
import yfinance as yf

### Download Stock Data from Yahoo Finance:

In [4]:
# Get Recent Business Date:
from datetime import datetime,timedelta
from pandas.tseries.offsets import BDay
recent_bus_day = (datetime.today()-BDay(1)).strftime("%Y-%m-%d")

# Auto-Update Time Interval
start=(datetime.today()-timedelta(days=3)).strftime("%Y-%m-%d")
end=datetime.today().strftime("%Y-%m-%d")
print(f"start date {start}")
print(f"end date {end}")
print(f"last business day {recent_bus_day}")

start date 2021-04-23
end date 2021-04-26
last business day 2021-04-23


In [5]:
# Dowload BRK Data from Yahoo
ticker = "BRK-A"
brk_yfinance = yf.download(ticker, start, end)
# Store in Dataframe
BRK=brk_yfinance["Adj Close"].to_frame(name="BRK")

[*********************100%***********************]  1 of 1 completed


In [6]:
# Dowload ETH Data from Yahoo
ticker = "ETH-USD"
eth_yfinance = yf.download(ticker, start, end)
# Store in Dataframe
ETH=eth_yfinance["Adj Close"].to_frame(name="ETH")

[*********************100%***********************]  1 of 1 completed


### Calculate BRK to ETH Ratio for the Last Business Day:

In [7]:
# Concat DataFrames
stock_data=pd.concat([BRK,ETH],axis='columns',join="inner").reset_index()
# Calculate and Extract Recent Ratio (the recent business day)
stock_data["ratio"]=stock_data["BRK"]/stock_data["ETH"]
ratio_df = stock_data.loc[stock_data["Date"]==recent_bus_day]
ratio_df

,Date,BRK,ETH,ratio
1,2021-04-23,408840.0,2363.586182,172.974442


In [8]:
# Store Recent BRK to ETH Ratio (the recent business day)
# Always Round Up!!
import math
num_ETH = math.ceil(ratio_df["ratio"])
share_price = int(ratio_df['BRK'].to_numpy())
ETH_price = int(ratio_df['ETH'].to_numpy())

print(f"BRK price of last business date: {share_price}")
print(f"ETH price of last business date: {ETH_price}")   
print(f"Required number of ETH: {num_ETH}")

BRK price of last business date: 408840
ETH price of last business date: 2363
Required number of ETH: 173


### Initialize & Deploy Solidity Contract: GoalDeployer

In [9]:
def initContract():
    with open(Path("GoalDeployer.json")) as json_file:
        abi = json.load(json_file)
    return w3.eth.contract(address="0x9b9a1a65D1bc7544F781e6B10C3749167CfAcB20", abi=abi)        

In [10]:
GoalDeployer = initContract()

### Sending Updated Data to Solidity Contract: GoalDeployer

In [11]:
def update_goal(date, share_price, ETH_price, ratio):
    tx_hash = GoalDeployer.functions.Update(date, share_price, ETH_price, ratio).transact(
        {"from": w3.eth.accounts[0]}
    )
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    print(f"Event log is updated for {date}")
    return receipt

In [12]:
update_goal(recent_bus_day, share_price, ETH_price, num_ETH)

CannotHandleRequest: Could not discover provider while making request: method:eth_accounts
params:()


In [13]:
def get_goal(date):
    price_filter = GoalDeployer.events.goal.createFilter(
        fromBlock="0x0", argument_filters={"date": date}
    )
    return price_filter.get_all_entries()

In [14]:
get_goal("2021-04-23")

CannotHandleRequest: Could not discover provider while making request: method:eth_newFilter
params:({'topics': ('0xc85bda2b06fad190ee52fa59cf5ca76af83ed14ff1a1ccd8b404541ca0b91885',), 'fromBlock': '0x0', 'toBlock': 'latest', 'address': ['0x9b9a1a65D1bc7544F781e6B10C3749167CfAcB20']},)


### Initialize Solidity Contract: BHCoinSale

In [15]:
def initContract():
    with open(Path("BHCoinSale.json")) as json_file:
        abi_2 = json.load(json_file)
    return w3.eth.contract(address="0x688bE037cd639254C082Ca1e1bB88F9817174183", abi=abi_2) 

In [16]:
BHCoinSale = initContract()

In [17]:
get_cap_status = BHCoinSale.functions.cap().call()
get_cap_status

CannotHandleRequest: Could not discover provider while making request: method:eth_call
params:({'to': '0x688bE037cd639254C082Ca1e1bB88F9817174183', 'data': '0x355274ea'}, 'latest')


In [25]:
get_capReached_status = BHCoinSale.functions.capReached().call()
get_capReached_status

CannotHandleRequest: Could not discover provider while making request: method:eth_call
params:({'to': '0x688bE037cd639254C082Ca1e1bB88F9817174183', 'data': '0x4f935945'}, 'latest')


In [34]:
get_goalReached_status = BHCoinSale.functions.goalReached().call()
get_goalReached_status 

CannotHandleRequest: Could not discover provider while making request: method:eth_call
params:({'to': '0x688bE037cd639254C082Ca1e1bB88F9817174183', 'data': '0x7d3d6522'}, 'latest')


In [ ]:
get_isOpen_status = BHCoinSale.functions.isOpen().call()
get_isOpen_status

In [ ]:
get_hasClosed_status = BHCoinSale.functions.hasClosed().call()
get_hasClosed_status

### Get Updated Sale Status from Solidity Contract: GoalDeployer

In [34]:
get_finalized_status = BHCoinSale.functions.finalized().call()
get_finalized_status

False

### Purchase The Share 

In [41]:
### Dummy to test if works....
get_finalized_status = True

In [42]:

def purchase(get_finalized_status): 
    
    if get_finalized_status == True: 
 
        #API endpoint URL
        #Init our account var
        account = api.get_account()

        #Should print 'ACTIVE'
        print(account.status)
        order = api.submit_order(symbol="BRK.A",
                                 qty="1",
                                 side="buy",
                                 type="limit",
                                 limit_price="180.15",
                                 time_in_force="day")
        return order    
    
    else: 
        time.sleep(3600)
        purchase
    
    

In [44]:
purchase(get_finalized_status)

ACTIVE


Order({   'asset_class': 'us_equity',
    'asset_id': '20801ccb-9347-4f2e-8860-2bfb77816275',
    'canceled_at': None,
    'client_order_id': '4b1e7acc-31c8-441f-9ee6-0272e1d8337c',
    'created_at': '2021-04-26T11:45:38.513713Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': 'c6a3cb1b-609b-42b1-af85-387a0eb60860',
    'legs': None,
    'limit_price': '180.15',
    'notional': None,
    'order_class': '',
    'order_type': 'limit',
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2021-04-26T11:45:38.5092Z',
    'symbol': 'BRK.A',
    'time_in_force': 'day',
    'trail_percent': None,
    'trail_price': None,
    'type': 'limit',
    'updated_at': '2021-04-26T11:45:38.513713Z'})